# **Tabular Reinforcement Learning**

# Monte Carlo methods on FrozenLake environment

## Non-Evaluables Practical Exercices

This is a non-evaluable practical exercise, but it is recommended that students complete it fully and individually, since it is an important part of the learning process.

The solution will be available, although it is not recommended that students consult the solution until they have completed the exercise.

## The FrozenLake environment

In this activity, we are going to implement the **Value Iteration** algorithm on [Frozen Lake](https://gymnasium.farama.org/environments/toy_text/frozen_lake/) environment.

Main characteristics:
- The game starts with the player at location [0,0] of the frozen lake grid world with the goal located at far extent of the world e.g. [3,3] for the 4x4 environment.
- Holes in the ice are distributed in set locations when using a pre-determined map or in random locations when a random map is generated.
- The player makes moves until they reach the goal or fall in a hole.
- The lake is slippery (unless disabled) so the player may move perpendicular to the intended direction sometimes (see _is_slippery_ param).

<img src="https://gymnasium.farama.org/_images/frozen_lake.gif" />

## Monte Carlo methods

In this section, we implement an estimation of the **optimal policy** using **Monte Carlo methods**, specifically we will study the **_On-policy every-visit MC control algorithm (for $\epsilon$-soft policies)_**.

<u>Question 1</u>: : **Implement the algorithm** using the following parameters:

- Number of episodes = 100,000
- Discount factor = 1

<u>Question 2</u>: : **Implement the epsilon decay factor** using the following equation and parameters:

- Initial epsilon = 1
- Epsilon decay factor (*epsilon decay*) = 0.999
- Update epsilon according to: $\textrm{max}(\epsilon · \epsilon_{\textrm{decay}}, 0.01)$

<u>Question 3</u>: Once you have coded the algorithm, try different **values for the hyperparameters** and comment the best ones (providing an empirical comparison):

- Number of episodes
- Initial epsilon
- Epsilon decay factor (*epsilon decay*)
- *discount factor* 

<u>Question 4</u>: Try to solve the same environment but using a _8 x 8_ grid (also in slippery mode):

> gym.make(ENV_NAME, desc=None, map_name="8x8", is_slippery=True)

In [1]:
import gymnasium as gym

# params
ENV_NAME = "FrozenLake-v1"
GAMMA = 0.9
TEST_EPISODES = 20

# definig the environment
env = gym.make(ENV_NAME, desc=None, map_name="4x4", is_slippery=False)

print("Action space is {} ".format(env.action_space))
print("Observation space is {} ".format(env.observation_space))

Action space is Discrete(4) 
Observation space is Discrete(16) 


In [2]:
def make_epsilon_greedy_policy(Q, epsilon, num_Actions):
    """
    Creates an epsilon-greedy policy based on a Q and epsilon action value function
    
    Args:
         Q: A dictionary whose correspondence is state -> action-values.
            Each value is a numpy array of length num_Actions (see below)
         epsilon: The probability of selecting a random action (float between 0 and 1).
         num_Actions: Number of actions in the environment (in the case of WIndyGridWorld, it is 4)
    
    Returns:
         A function that takes the observation as an argument and returns as a result
         the probabilities of each action as a numpy array of length num_Actions.
    """
    

def mc_control_on_policy_epsilon_greedy(env, num_episodes, discount=1.0, epsilon=0.1, epsilon_decay = 0.9):
    """
    Control by Monte Carlo methods using Epsilon-Greedy policies
    Find an epsilon-greedy policy.
    
    Args:
         env: Gymnasium environment.
         num_episodes: Number of episodes in the sample.
         discount: discount factor.
         epsilon: The probability of selecting a random action (float between 0 and 1)
    
    Returns:
         A tuple (Q, policy).
         Q: A dictionary whose correspondence is state -> action-values.
         policy: A function that takes the observation as an argument and returns as a result
                 the probabilities of each action
    """


<div class="alert alert-block alert-danger">
<strong>Solution</strong>
</div>

In [3]:
import numpy as np
from collections import defaultdict
import sys
from tensorboardX import SummaryWriter

# writer
writer = SummaryWriter(comment="-mc_control")


def make_epsilon_greedy_policy(Q, epsilon, num_Actions):
    """
    Creates an epsilon-greedy policy based on a Q and epsilon action value function
    
    Args:
         Q: A dictionary whose correspondence is state -> action-values.
            Each value is a numpy array of length num_Actions (see below)
         epsilon: The probability of selecting a random action (float between 0 and 1).
         num_Actions: Number of actions in the environment. (in the case of WIndyGridWorld it is 4)
    
    Returns:
         A function that takes the observation as an argument and returns as a result
         the probabilities of each action as a numpy array of length num_Actions.
    """
    def policy_fn(observation):
        A = np.ones(num_Actions, dtype=float) * epsilon / num_Actions
        best_action = np.argmax(Q[observation])
        A[best_action] += (1.0 - epsilon)

        return A
    
    return policy_fn


def mc_control_on_policy_epsilon_greedy(env, num_episodes, discount=1.0, epsilon=0.1, epsilon_decay=0.9):
    """
    Control by Monte Carlo methods using Epsilon-Greedy policies
    Find an epsilon-greedy policy.
    
    Args:
         env: Gymnasium environment.
         num_episodes: Number of episodes in the sample.
         discount: discount factor.
         epsilon: The probability of selecting a random action (float between 0 and 1)
    
    Returns:
         A tuple (Q, policy).
         Q: A dictionary whose correspondence is state -> action-values.
         policy: A function that takes the observation as an argument and returns as a result
                 the probabilities of each action
    """
    
    # We store the sum and number of returns for each state to calculate the average. 
    # We could use an array to store all the returns, but it is inefficient in terms of memory.
    returns_sum = defaultdict(float)
    returns_count = defaultdict(float)
    
    # The Q action value function.
    # A nested dictionary whose correspondence is state -> (action -> action-value).
    # Initially we initialize it to zero
    Q = defaultdict(lambda: np.zeros(env.action_space.n))
    
    # Rewards
    y = np.zeros(num_episodes, dtype=np.float16)
    
    for i_episode in range(num_episodes):
        # The policy we are following
        policy = make_epsilon_greedy_policy(Q, epsilon, env.action_space.n)
        
        # We update epsilon
        epsilon = max(epsilon * epsilon_decay, 0.01)

        # We generate an episode and store it
        # An episode is an array of tuples (state, action, reward)
        episode = []
        state, _ = env.reset()
        done = False
        total_reward = 0
        while not done:
            probs = policy(state)
            action = np.random.choice(np.arange(len(probs)), p=probs)
            next_state, reward, terminated, truncated, _ = env.step(action)
            done = terminated or truncated
            episode.append((state, action, reward))
            total_reward += reward
            if done:
                break
            state = next_state

        y[i_episode] = total_reward
        idx = 0
        
        # We visited all the states of the episode
        G = 0
        for i in range(len(episode)-1, -1, -1):
            x = episode[i]
            state = x[0]
            action = x[1]
            reward = x[2]
            sa_pair = (state, action)
            # We calculate the return of each state
            G = reward + discount * G
            # We calculate the average return for this state across all sampled episodes.   
            returns_sum[sa_pair] += G
            returns_count[sa_pair] += 1.0
            Q[state][action] = returns_sum[sa_pair] / returns_count[sa_pair]
            idx = idx + 1
        
        # write to summary
        writer.add_scalar("reward", total_reward, i_episode)
        
        # We print which episode we are in, useful for debugging.
        if i_episode % 100 == 0 and i_episode > 0:
            print("\rEpisode {:8d}/{:8d} - Average reward {:.2f}".format(i_episode, num_episodes, np.average(y[(i_episode-100):i_episode])), end="")
            sys.stdout.flush()
            
        # The policy is implicitly improved by changing the values of Q
    
    return Q, policy

In [4]:
# Execute
Q, policy = mc_control_on_policy_epsilon_greedy(env, num_episodes=50000, discount=1, epsilon=1, epsilon_decay=0.999)

Episode    49900/   50000 - Average reward 0.99

In [5]:
# execution of an episode following the optimal policy
def execute_episode_MC(q, env, debug=False):
    obs, _ = env.reset()
    t, total_reward, done = 0, 0, False

    if debug:
        print("Obs initial: {} ".format(obs))

    while True: 
        # Choose an action following the optimal policy (no epsilon-greedy)
        arr = np.array(q[obs])
        action = arr.argmax()
       
        # Execute the action and wait for the response from the environment
        new_obs, reward, terminated, truncated, info = env.step(action)
        done = terminated or truncated
        obs = new_obs
        if debug:
            print("Action: {} -> Obs: {} and reward: {}".format(action, obs, reward))

        total_reward += reward
        t += 1
        if done:
            break
   
    print("Episode finished after {} timesteps and reward was {} ".format(t, total_reward))
    return total_reward

In [6]:
execute_episode_MC(Q, env, debug=True)

Obs initial: 0 
Action: 1 -> Obs: 4 and reward: 0.0
Action: 1 -> Obs: 8 and reward: 0.0
Action: 2 -> Obs: 9 and reward: 0.0
Action: 1 -> Obs: 13 and reward: 0.0
Action: 2 -> Obs: 14 and reward: 0.0
Action: 2 -> Obs: 15 and reward: 1.0
Episode finished after 6 timesteps and reward was 1.0 


1.0

In [7]:
from tensorboardX import SummaryWriter
writer = SummaryWriter(comment="-mc_test")

total_reward = 0.0
for iter_no in range(TEST_EPISODES):
    reward = execute_episode_MC(Q, env)
    writer.add_scalar("reward", reward, iter_no)
    total_reward += reward

avg_reward = total_reward / TEST_EPISODES
print("Average reward is {} ({} episodes)".format(avg_reward, TEST_EPISODES))
if reward > 0.80:
    print("Environment solved!")

writer.close()

Episode finished after 6 timesteps and reward was 1.0 
Episode finished after 6 timesteps and reward was 1.0 
Episode finished after 6 timesteps and reward was 1.0 
Episode finished after 6 timesteps and reward was 1.0 
Episode finished after 6 timesteps and reward was 1.0 
Episode finished after 6 timesteps and reward was 1.0 
Episode finished after 6 timesteps and reward was 1.0 
Episode finished after 6 timesteps and reward was 1.0 
Episode finished after 6 timesteps and reward was 1.0 
Episode finished after 6 timesteps and reward was 1.0 
Episode finished after 6 timesteps and reward was 1.0 
Episode finished after 6 timesteps and reward was 1.0 
Episode finished after 6 timesteps and reward was 1.0 
Episode finished after 6 timesteps and reward was 1.0 
Episode finished after 6 timesteps and reward was 1.0 
Episode finished after 6 timesteps and reward was 1.0 
Episode finished after 6 timesteps and reward was 1.0 
Episode finished after 6 timesteps and reward was 1.0 
Episode fi